In [1]:
import pandas as pd

data1 = r"..\data\raw\Steam_2024_bestRevenue_1500.csv"
data2 = r"..\data\raw\game_api_data.csv"
data3 = r"..\data\raw\final_merged_steamdata.csv"

data1_df = pd.read_csv(data1)
data2_df = pd.read_csv(data2, sep=';')
data3_df = pd.read_csv(data3)

In [2]:
def merge_dataframes(df1, df2, df3):
    df_merge1 = pd.merge(df1, df2, left_on="steamId", right_on="steam_id", how="left")
    df_merge2 = pd.merge(df_merge1, df3, left_on="steamId", right_on="steamId", how="left")
    return df_merge2

In [3]:
merged_df = merge_dataframes(data1_df, data2_df, data3_df)

In [16]:
col_list = ['steamId', 'name_x', 'required_age', 'required age']

df_test = merged_df[col_list].copy()

df_test['required age'] = df_test['required age'].fillna(df_test['required_age'])
df_test['required age'] = df_test['required age'].fillna(0)
df_test['required_age'] = df_test['required_age'].fillna(df_test['required age'])

df_test.loc[df_test['required_age'] == 0, 'required_age'] = df_test['required age']

df_test = df_test[df_test['required_age'] != df_test['required age']]



df_test
#missing_values_df = df_test[df_test.isna().any(axis=1)]

#df_test[df_test['developers_x_x'].isna()]

,steamId,name_x,required_age,required age
0,2315690,WWE 2K24,16.0,13.0
15,1840080,Homeworld 3,7.0,10.0
194,1880470,The Inquisitor,0.0,16.0
283,1674920,Sovereign Syndicate,0.0,17.0
432,2414550,MADiSON VR,16.0,17.0
484,655740,Tallowmere 2: Curse of the Kittens,0.0,13.0
587,1214650,SOUTH PARK: SNOW DAY!,0.0,17.0
606,1310410,Alone in the Dark,0.0,17.0
1198,2662630,The Elder Scrolls Online: Gold Road,18.0,17.0
1250,1519090,Welcome to ParadiZe,18.0,17.0


In [42]:
def clean_cols(df):
    new_df = df.dropna(axis=1, thresh=len(df) - 500)
    new_df = new_df.rename(columns={'name_x': 'name'})
    new_df['genres_x'] = df['genres_x'].fillna(df['genres_y'])
    new_df = new_df.rename(columns={'genres_x': 'genres'})
    new_df['publishers_x_x'] = new_df['publishers_x_x'].fillna(new_df['publishers_y_y'])
    new_df['publishers_x_x'] = new_df['publishers_x_x'].str.replace(",", ", ").str.replace("  ", " ")
    new_df['developers_x_x'] = new_df['developers_x_x'].str.replace(",", ", ").str.replace("  ", " ")
    new_df['developers_x_x'] = new_df['developers_x_x'].fillna(new_df['developers_y_y'])

    new_df['required age'] = new_df['required age'].fillna(0)
    new_df['required_age'] = new_df['required_age'].fillna(new_df['required age'])
    new_df.loc[new_df['required_age'] == 0, 'required_age'] = new_df['required age'] # This line is
    
    col_list = ['name', 'releaseDate_x', 'copiesSold_x', 'price_x_x', 'revenue_x',
       'avgPlaytime_x', 'reviewScore_x', 'publisherClass_x', 'publishers_x_x',
       'developers_x_x', 'steamId', 'required_age',
       'is_free', 'description', 'languages', 
       'price_y_x', 'genres',
       'categories_x',
       'estimated owners', 'peak ccu',
       'about the game', 
       'metacritic score', 'user score', 'positive',
       'negative', 'recommendations',
       'average playtime forever', 'average playtime two weeks',
       'median playtime forever', 'median playtime two weeks'
       ]
    new_df = new_df[col_list]
    new_df['releaseDate_x'] = pd.to_datetime(new_df['releaseDate_x'], format='%d-%m-%Y', errors='coerce')
    new_df = new_df.rename(columns={'releaseDate_x': 'release_date', 'revenue_x': 'revenue', 'avgPlaytime_x': 'avgPlaytime', 'publisherClass_x': 'publisherClass'})
    new_df = new_df.rename(columns={'copiesSold_x': 'copies_sold', 'price_x_x':'price_USD', 'price_y_x': 'price_local', 'reviewScore_x': "review_score" })
    new_df = new_df.rename(columns={'publishers_x_x': 'publishers', 'developers_x_x': 'developers', 'categories_x': 'categories'})

    cols = [
        'steamId', 'name',
        'release_date',
        'copies_sold',
        'price_USD', 'price_local',
        'revenue',
        'avgPlaytime',
        'publisherClass',
        'publishers',
        'developers',
        'required_age',    
        'is_free',
        'description', 'about the game',
        'languages',
        'genres',
        'categories',
        'estimated owners','peak ccu',        
        'metacritic score', 'review_score', 'user score',
        'positive', 'negative',
        'recommendations',
        'average playtime forever', 'average playtime two weeks', 'median playtime forever', 'median playtime two weeks']

    new_df = new_df[cols]
    return new_df

In [43]:
filtered_df = clean_cols(merged_df)

filtered_df.isna().sum()

steamId                         0
name                            0
release_date                    0
copies_sold                     0
price_USD                       0
price_local                    90
revenue                         0
avgPlaytime                     0
publisherClass                  0
publishers                      0
developers                      0
required_age                    0
is_free                         2
description                     2
about the game                229
languages                       2
genres                          2
categories                      2
estimated owners              222
peak ccu                      222
metacritic score              222
review_score                    0
user score                    222
positive                      222
negative                      222
recommendations               222
average playtime forever      222
average playtime two weeks    222
median playtime forever       222
median playtim